In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import Levenshtein as lev
from fuzzywuzzy import fuzz, process

In [118]:
clientes_2020 = pd.read_csv('Clientes_Dic2020.csv',sep=';')
clientes = pd.read_csv('Clientes.csv',sep=';')

In [195]:
clientes.tail()

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y,col10
2379,2668,Córdoba,MARIA STELLA RIVAS,CALLE PUBLICA SANTA ELENA COLONIA TIROLESA - ...,4902756,23.0,COLONIA TIROLESA,"-64,7419272","-31,25881725",NaN
2380,2669,Córdoba,ANA GABRIELA RIVAS,CALLE PUBLICA S/NÂº LA ARCADIA,153580992,28.0,LA ARCADIA,"-64,7696886","-31,53338396",NaN
2381,2670,Córdoba,JAVIER ALONSO RIVEIRO,AVENIDA LIBERTADOR SAN MARTIN S/NÂº BOUWER,4993041,57.0,BOUWER,"-64,19298343","-31,55786125",NaN
2382,2671,Córdoba,JUAN MANUEL RODRIGUEZ,CALLE PUBLICA S/NÂ° EL GATEADO BARRIO EL GATE...,152768918,36.0,EL GATEADO,"-64,11681227","-31,32123448",NaN
2383,2672,Córdoba,JULIO CESAR RODRIGUEZ,CALLE PUBLICA S/NÂº EL VERGEL P/COLONIA TIROLESA,156149888,42.0,EL VERGEL,"-64,7463455","-31,25832604",NaN


In [196]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2384 entries, 0 to 2383
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 2384 non-null   int64  
 1   Provincia          2359 non-null   object 
 2   Nombre_y_Apellido  2365 non-null   object 
 3   Domicilio          2341 non-null   object 
 4   Telefono           2354 non-null   object 
 5   Edad               2383 non-null   float64
 6   Localidad          2359 non-null   object 
 7   X                  2384 non-null   object 
 8   Y                  2384 non-null   object 
 9   col10              0 non-null      float64
dtypes: float64(2), int64(1), object(7)
memory usage: 186.4+ KB


In [122]:
clientes = pd.concat([clientes, clientes_2020], axis=0)

In [125]:
clientes = clientes.sort_values(by='ID')
clientes_2020 = clientes_2020.sort_values(by='ID')

In [123]:
clientes = clientes.sort_index()

In [126]:
clientes.head(22)

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y,col10
0,1,Buenos Aires,HEBER JONI SANTANA,LAS HERAS Y BAT. 24 DE FEBRERO 4150 RINCON DE...,42-5161,58.0,LOMA VERDE,"-58,81850307","-34,30997088",NaN
1,2,Buenos Aires,ANA SAPRIZA,PUEYRREDON Y DUPUY RUTA3 KM 52.500 S/N BÂº LO...,49-7578,61.0,SANTA ROSA,"-58,73073751","-34,93908311",NaN
2,3,Buenos Aires,FERNANDO LUIS SARALEGUI,CALDERON DE LA BARCA 498,49-3435,15.0,TORRES,"-59,12794068","-34,43082199",NaN
3,4,Buenos Aires,MANUELA SARASOLA,"RUTA 36 KM 45, 500 S/N EL PELIGRO",49-2883,29.0,RUTA SOL,"-58,14393954","-34,92052706",NaN
4,5,Buenos Aires,MARIO RAÚL SARASUA,492 Y 186 S/N COLONIA URQUIZA,491-4608,34.0,JOSE MELCHOR ROMERO,"-58,89381","-34,9444471",NaN
5,6,Buenos Aires,PEDRO JESUS SARAVIA,"RUTA 2 - KM 44, 500 S/N EL PELIGRO",49-2350,18.0,RUTA SOL,"-58,11226426","-35,786216",NaN
6,7,Buenos Aires,JORGE SARAVIA,VILLARROEL RUTA 3 KM 46500 S/N BARRIO SAN MAR...,49-5306,21.0,VIRREY DEL PINO,"-58,70894814","-34,86070786",NaN
7,8,Buenos Aires,CARLOS JOSE SARAZOLA,ISLA SANTIAGO S/N ISLA SANTIAGO,623-9935,40.0,ISLA SANTIAGO,"-57,88154205","-34,8350313",NaN
8,9,Buenos Aires,OSCAR LUIS SARLO,GARCILAZO DE LA VEGA Y SAN MARTIN S/N SANTA ROSA,49-7576,18.0,SANTA ROSA,"-58,75008438","-34,97534955",NaN
9,10,Buenos Aires,JOSE ADOLFO SARMIENTO,SEGUNDO SOMBRA E/MAGALLANES Y P. GALDO S/N SA...,49-7565,58.0,SANTA ROSA,"-58,75203716","-34,95142843",NaN


In [127]:
clientes = clientes.drop(columns='col10')

In [63]:
clientes[clientes['X'].isnull()]

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y,col10
69,830,Buenos Aires,MARIANELLA CARDENAS GONZALEZ,NaN,421-2978,19.0,LA PLATA,NaN,NaN,NaN
93,1612,Buenos Aires,ALEJANDRO HERNANDEZ,NaN,4651-3232,44.0,SAN JUSTO,NaN,NaN,NaN
255,2557,Córdoba,DANIEL HONORIO BARRIOS,NaN,NaN,17.0,CORDOBA,NaN,NaN,NaN


In [202]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2960 entries, 0 to 287
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 2960 non-null   int64  
 1   Provincia          2935 non-null   object 
 2   Nombre_y_Apellido  2931 non-null   object 
 3   Domicilio          2911 non-null   object 
 4   Telefono           2922 non-null   object 
 5   Edad               2959 non-null   float64
 6   Localidad          2935 non-null   object 
 7   X                  2954 non-null   object 
 8   Y                  2952 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 231.2+ KB


In [10]:
clientes = clientes.fillna('N/D')

In [11]:
clientes[clientes['ID'] == 1803]

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y
1708,1803,Buenos Aires,DANIELA RAMOS,RIO PARANA DE LAS PALMAS KM. 117,5 S/N,N/D,21,"ISLAS,-59","9145931,-34"


In [7]:
localidad = pd.read_csv('Localidades.csv')

In [8]:
gasto = pd.read_csv('Gasto.csv')

In [157]:
gasto.tail()

,IdGasto,IdSucursal,IdTipoGasto,Fecha,Monto
8635,8636,23,4,2020-12-01,1180.27
8636,8637,24,1,2020-12-01,1152.15
8637,8638,24,2,2020-12-01,305.98
8638,8639,24,3,2020-12-01,104.32
8639,8640,24,4,2020-12-01,1156.31


In [9]:
gasto['Fecha'] = pd.to_datetime(gasto['Fecha'])

In [156]:
gasto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8640 entries, 0 to 8639
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   IdGasto      8640 non-null   int64  
 1   IdSucursal   8640 non-null   int64  
 2   IdTipoGasto  8640 non-null   int64  
 3   Fecha        8640 non-null   object 
 4   Monto        8640 non-null   float64
dtypes: float64(1), int64(3), object(1)
memory usage: 337.6+ KB


In [109]:
compras = pd.read_csv('Compra.csv')

In [110]:
compras.head()

,IdCompra,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor
0,1,1/30/2015,2015,1,201501,42832,13,560.51,12
1,2,1/30/2015,2015,1,201501,42833,11,497.58,7
2,3,1/30/2015,2015,1,201501,42834,1,588.50,6
3,4,1/30/2015,2015,1,201501,42835,9,567.66,14
4,5,1/30/2015,2015,1,201501,42839,14,231.31,2


In [152]:
compras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11539 entries, 0 to 11538
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   IdCompra       11539 non-null  int64  
 1   Fecha          11539 non-null  object 
 2   Fecha_Año      11539 non-null  int64  
 3   Fecha_Mes      11539 non-null  int64  
 4   Fecha_Periodo  11539 non-null  int64  
 5   IdProducto     11539 non-null  int64  
 6   Cantidad       11539 non-null  int64  
 7   Precio         11172 non-null  float64
 8   IdProveedor    11539 non-null  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 811.5+ KB


In [11]:
compras = compras.drop(['Fecha_Año','Fecha_Mes','Fecha_Periodo'], axis=1)

In [12]:
compras['Fecha'] = pd.to_datetime(compras['Fecha'])

In [13]:
compras['Cantidad'] = compras['Cantidad'].astype('float')

In [14]:
compras['Precio'] = compras['Precio'].fillna(0)

In [15]:
venta_dic = pd.read_csv('Venta_Dic2020.csv')
ventas = pd.read_csv('Venta.csv')

In [16]:
venta_dic.head()

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
0,47612,2020-12-01,2020-12-08,1,1667,13,1674,42945,3210.0,3.0
1,47613,2020-12-01,2020-12-09,2,979,1,1517,42988,598.0,2.0
2,47614,2020-12-01,2020-12-04,2,708,1,1056,42883,498.0,2.0
3,47615,2020-12-01,2020-12-05,2,640,1,1674,42922,741.0,2.0
4,47616,2020-12-01,2020-12-07,1,524,14,1076,42933,1154.0,1.0


In [17]:
venta_dic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   IdVenta        465 non-null    int64  
 1   Fecha          465 non-null    object 
 2   Fecha_Entrega  465 non-null    object 
 3   IdCanal        465 non-null    int64  
 4   IdCliente      465 non-null    int64  
 5   IdSucursal     465 non-null    int64  
 6   IdEmpleado     465 non-null    int64  
 7   IdProducto     465 non-null    int64  
 8   Precio         461 non-null    float64
 9   Cantidad       457 non-null    float64
dtypes: float64(2), int64(6), object(2)
memory usage: 36.5+ KB


In [18]:
venta_dic['Fecha'] = pd.to_datetime(venta_dic['Fecha'])
venta_dic['Fecha_Entrega'] = pd.to_datetime(venta_dic['Fecha_Entrega'])
ventas['Fecha'] = pd.to_datetime(ventas['Fecha'])
ventas['Fecha_Entrega'] = pd.to_datetime(ventas['Fecha_Entrega'])

In [19]:
ventas = pd.concat([ventas, venta_dic], axis=0)

In [20]:
ventas.tail()

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
460,48237,2020-12-30,2021-01-02,1,1368,20,3186,42880,560.00,2.0
461,48238,2020-12-30,2021-01-02,2,354,23,3504,42883,498.00,1.0
462,48239,2020-12-30,2021-01-06,2,2400,25,3622,42814,369.82,2.0
463,48240,2020-12-30,2021-01-03,2,1803,30,3864,42998,529.00,3.0
464,48241,2020-12-30,2021-01-02,2,226,31,3205,42810,1282.82,2.0


In [21]:
ventas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46645 entries, 0 to 464
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   IdVenta        46645 non-null  int64         
 1   Fecha          46645 non-null  datetime64[ns]
 2   Fecha_Entrega  46645 non-null  datetime64[ns]
 3   IdCanal        46645 non-null  int64         
 4   IdCliente      46645 non-null  int64         
 5   IdSucursal     46645 non-null  int64         
 6   IdEmpleado     46645 non-null  int64         
 7   IdProducto     46645 non-null  int64         
 8   Precio         45721 non-null  float64       
 9   Cantidad       45753 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(6)
memory usage: 3.9 MB


In [133]:
ventas[ventas['Precio'] >= 42839 ]

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
131,132,2015-03-11,2015-03-13,3,2837,13,1674,42949,201500.0,1.0
164,165,2020-04-01,2020-04-05,2,2823,13,1674,42890,51500.0,2.0
237,238,2016-11-09,2016-11-12,1,2838,13,1516,42942,274900.0,2.0
533,534,2019-06-03,2019-06-08,1,3237,13,1330,42798,155628.0,2.0
603,604,2016-07-30,2016-08-04,3,2777,13,1330,42911,178900.0,1.0
...,...,...,...,...,...,...,...,...,...,...
44996,45972,2020-09-26,2020-10-04,2,305,23,3119,42853,72700.0,3.0
45189,46230,2020-10-03,2020-10-06,2,1055,21,3363,42875,56000.0,2.0
45364,46486,2020-10-21,2020-10-23,2,2229,10,2557,43029,109400.0,2.0
46113,47521,2020-11-27,2020-12-03,1,8,1,1968,42773,870364.0,1.0


In [22]:
productos = ventas[(ventas['Cantidad'] == 1) & (ventas['Precio'] != np.nan)]

In [141]:
productos.count()

IdVenta          11011
Fecha            11011
Fecha_Entrega    11011
IdCanal          11011
IdCliente        11011
IdSucursal       11011
IdEmpleado       11011
IdProducto       11011
Precio           10788
Cantidad         11011
dtype: int64

In [23]:
productos = productos.drop(['IdVenta','Fecha','Fecha_Entrega','IdCanal','IdCliente','IdSucursal','IdEmpleado'],axis=1)

In [24]:
productos = productos.sort_values('IdProducto')
productos = productos.sort_values('Precio')

In [146]:
productos.head()

,IdProducto,Precio,Cantidad
12443,42925,3.0,1.0
21452,42982,13.0,1.0
35497,42981,52.0,1.0
29794,42926,79.0,1.0
42955,42927,91.0,1.0


In [25]:
productos = productos.drop_duplicates(subset='IdProducto',keep='first')

In [134]:
productos[productos['IdProducto'] == 42949]

,IdProducto,Precio,Cantidad
39960,42949,2015.0,1.0


In [148]:
pd.merge(ventas, productos['Precio'],how='left')

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2.0
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3.0
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1.0
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2.0
4,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2.0
...,...,...,...,...,...,...,...,...,...,...
99802,48238,2020-12-30,2021-01-02,2,354,23,3504,42883,498.00,1.0
99803,48238,2020-12-30,2021-01-02,2,354,23,3504,42883,498.00,1.0
99804,48239,2020-12-30,2021-01-06,2,2400,25,3622,42814,369.82,2.0
99805,48240,2020-12-30,2021-01-03,2,1803,30,3864,42998,529.00,3.0


In [139]:
productos[productos['IdProducto'] == 42949]

,IdProducto,Precio,Cantidad
39960,42949,2015.0,1.0


In [114]:
ventas.head()

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2.0
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3.0
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1.0
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2.0
4,5,2017-11-22,2017-11-25,2,678,13,1674,42825,554.18,3.0


In [156]:
localidad = pd.read_csv('Localidades.csv')

In [149]:
localidad.head()

,categoria,centroide_lat,centroide_lon,departamento_id,departamento_nombre,fuente,id,localidad_censal_id,localidad_censal_nombre,municipio_id,municipio_nombre,nombre,provincia_id,provincia_nombre
0,Localidad simple,-35.033073,-60.280620,6021.0,Alberti,INDEC,6021010000,6021010,Alberti,60021.0,Alberti,ALBERTI,6,Buenos Aires
1,Localidad simple,-34.868119,-60.393971,6021.0,Alberti,INDEC,6021020000,6021020,Coronel Seguí,60021.0,Alberti,CORONEL SEGUI,6,Buenos Aires
2,Componente de localidad compuesta,-35.068014,-60.402597,6021.0,Alberti,INDEC,6021030000,6021030,Mechita,60021.0,Alberti,MECHITA,6,Buenos Aires
3,Localidad simple,-35.124382,-60.220061,6021.0,Alberti,INDEC,6021040000,6021040,Pla,60021.0,Alberti,PLA,6,Buenos Aires
4,Localidad simple,-35.109618,-60.070551,6021.0,Alberti,INDEC,6021050000,6021050,Villa Grisolía,60021.0,Alberti,VILLA GRISOLIA,6,Buenos Aires


In [28]:
localidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4142 entries, 0 to 4141
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   categoria                4142 non-null   object 
 1   centroide_lat            4142 non-null   float64
 2   centroide_lon            4142 non-null   float64
 3   departamento_id          4141 non-null   float64
 4   departamento_nombre      4141 non-null   object 
 5   fuente                   4142 non-null   object 
 6   id                       4142 non-null   int64  
 7   localidad_censal_id      4142 non-null   int64  
 8   localidad_censal_nombre  4142 non-null   object 
 9   municipio_id             3599 non-null   float64
 10  municipio_nombre         3648 non-null   object 
 11  nombre                   4142 non-null   object 
 12  provincia_id             4142 non-null   int64  
 13  provincia_nombre         4142 non-null   object 
dtypes: float64(4), int64(3),

In [150]:
provincia = localidad

In [151]:
provincia = provincia.drop(['categoria','centroide_lat','centroide_lon','departamento_id','departamento_nombre',
                            'id','localidad_censal_id','localidad_censal_nombre','municipio_id','municipio_nombre'
                            ,'nombre'],axis=1)

In [152]:
provincia.head()

,fuente,provincia_id,provincia_nombre
0,INDEC,6,Buenos Aires
1,INDEC,6,Buenos Aires
2,INDEC,6,Buenos Aires
3,INDEC,6,Buenos Aires
4,INDEC,6,Buenos Aires


In [153]:
provincia = provincia.drop_duplicates(subset='provincia_id',keep='first')

In [157]:
localidad = localidad.drop(['provincia_nombre'],axis=1)

In [161]:
localidad.head()

,categoria,centroide_lat,centroide_lon,departamento_id,fuente,id,localidad_censal_id,localidad_censal_nombre,municipio_id,municipio_nombre,nombre,provincia_id
0,Localidad simple,-35.033073,-60.280620,6021.0,INDEC,6021010000,6021010,Alberti,60021.0,Alberti,ALBERTI,6
1,Localidad simple,-34.868119,-60.393971,6021.0,INDEC,6021020000,6021020,Coronel Seguí,60021.0,Alberti,CORONEL SEGUI,6
2,Componente de localidad compuesta,-35.068014,-60.402597,6021.0,INDEC,6021030000,6021030,Mechita,60021.0,Alberti,MECHITA,6
3,Localidad simple,-35.124382,-60.220061,6021.0,INDEC,6021040000,6021040,Pla,60021.0,Alberti,PLA,6
4,Localidad simple,-35.109618,-60.070551,6021.0,INDEC,6021050000,6021050,Villa Grisolía,60021.0,Alberti,VILLA GRISOLIA,6


In [143]:
departamentos = localidad.drop(['categoria','centroide_lat','centroide_lon',
                            'id','localidad_censal_id','localidad_censal_nombre','municipio_id','municipio_nombre'
                            ,'nombre'],axis=1)

In [159]:
localidad = localidad.drop(['departamento_nombre'],axis=1)

In [166]:
departamentos = departamentos.drop_duplicates(subset='departamento_id', keep='first')

In [167]:
departamentos

,departamento_id,departamento_nombre,fuente
0,6021.0,Alberti,INDEC
7,6028.0,Almirante Brown,INDEC
19,6035.0,Avellaneda,INDEC
28,6042.0,Ayacucho,INDEC
32,6049.0,Azul,INDEC
...,...,...,...
4126,2091.0,Comuna 13,INDEC
4129,2098.0,Comuna 14,INDEC
4130,2105.0,Comuna 15,INDEC
4138,94008.0,Río Grande,INDEC


In [164]:
municipios = localidad.drop(['categoria','centroide_lat','centroide_lon',
                            'id','localidad_censal_id','localidad_censal_nombre','provincia_id','departamento_id'
                            ,'nombre'],axis=1)

In [170]:
municipios = municipios.drop_duplicates(subset='municipio_id',keep='first')

In [173]:
municipios.head()

,fuente,municipio_id,municipio_nombre
0,INDEC,60021.0,Alberti
7,INDEC,60028.0,Almirante Brown
19,INDEC,60035.0,Avellaneda
28,INDEC,60042.0,Ayacucho
32,INDEC,60049.0,Azul


In [171]:
localidad = localidad.drop(['municipio_nombre'],axis=1)

In [172]:
localidad.head()

,categoria,centroide_lat,centroide_lon,departamento_id,fuente,id,localidad_censal_id,localidad_censal_nombre,municipio_id,nombre,provincia_id
0,Localidad simple,-35.033073,-60.280620,6021.0,INDEC,6021010000,6021010,Alberti,60021.0,ALBERTI,6
1,Localidad simple,-34.868119,-60.393971,6021.0,INDEC,6021020000,6021020,Coronel Seguí,60021.0,CORONEL SEGUI,6
2,Componente de localidad compuesta,-35.068014,-60.402597,6021.0,INDEC,6021030000,6021030,Mechita,60021.0,MECHITA,6
3,Localidad simple,-35.124382,-60.220061,6021.0,INDEC,6021040000,6021040,Pla,60021.0,PLA,6
4,Localidad simple,-35.109618,-60.070551,6021.0,INDEC,6021050000,6021050,Villa Grisolía,60021.0,VILLA GRISOLIA,6


In [94]:
proveedor = pd.read_csv('Proveedores.csv', encoding='latin1')

In [98]:
proveedor.head()

,IDProveedor,Nombre,Direccion,Ciudad,Provincia,Pais,Departamento
0,1,NaN,bereguestein 36,avellaneda,buenos aires,argentina,avellaneda
1,2,San Cirano,alberti 965,coronel brandsen,buenos aires,argentina,brandsen
2,3,Bell S.A.,moreno 331,campana,buenos aires,argentina,campana
3,4,Rivero Insumos,pte. peron 3872,valentin alsina,buenos aires,argentina,lanus
4,5,Laprida Computacion,avda. pedro pereyra 1339,laprida,buenos aires,argentina,laprida


In [96]:
proveedor = proveedor.rename({'Address':'Direccion','City':'Ciudad','State':'Provincia','Country':'Pais',
                                'departamen':'Departamento'},axis=1)

In [97]:
proveedor['Direccion']=proveedor.Direccion.str.lower()
proveedor['Ciudad']=proveedor.Ciudad.str.lower()
proveedor['Provincia']=proveedor.Provincia.str.lower()
proveedor['Pais']=proveedor.Pais.str.lower()
proveedor['Departamento']=proveedor.Departamento.str.lower()

In [165]:
proveedor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   IDProveedor  14 non-null     int64 
 1   Nombre       12 non-null     object
 2   Address      14 non-null     object
 3   City         14 non-null     object
 4   State        14 non-null     object
 5   Country      14 non-null     object
 6   departamen   14 non-null     object
dtypes: int64(1), object(6)
memory usage: 912.0+ bytes


In [174]:
sucursal = pd.read_csv('Sucursales.csv', sep=';')

In [175]:
sucursal.head()

,ID,Sucursal,Direccion,Localidad,Provincia,Latitud,Longitud
0,1,Cabildo,Av. Cabildo 1342,Ciudad de Buenos Aires,Ciudad de Buenos Aires,"-34,5678060","-58,4495720"
1,2,Palermo 1,Guatemala 5701,CABA,CABA,"-34,5790350","-58,4335660"
2,3,Palermo 2,Gral. Lucio Norberto Mansilla 2668,CABA,C deBuenos Aires,"-34,5959660","-58,4051500"
3,4,Corrientes,Av. Corrientes 2352,Ciudad de Buenos Aires,Bs As,"-34,6046850","-58,3987640"
4,5,Almagro,Venezuela 3650,Capital,Bs.As.,"-34,6173080","-58,4161790"


In [171]:
sucursal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         31 non-null     int64 
 1   Sucursal   31 non-null     object
 2   Direccion  31 non-null     object
 3   Localidad  31 non-null     object
 4   Provincia  31 non-null     object
 5   Latitud    31 non-null     object
 6   Longitud   31 non-null     object
dtypes: int64(1), object(6)
memory usage: 1.8+ KB


In [137]:
clientes.groupby('Localidad').count()

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,X,Y
Localidad,,,,,,,,
11 DE SEPTIEMBRE,1,1,1,1,1,1,1,1
21,1,1,1,1,1,0,1,1
2º SECCION DE ISLAS,1,1,1,1,1,1,1,1
3º SECCION DE ISLAS,1,1,1,1,1,1,1,1
9 DE ABRIL,10,10,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...
WILDE,11,11,11,11,11,11,11,11
WILLIAM C MORRIS,6,6,6,6,6,6,6,6
ZARATE,22,22,22,22,21,22,22,22


In [120]:
sucursal.groupby('Provincia').count()

,ID,Sucursal,Direccion,Localidad,Latitud,Longitud
Provincia,,,,,,
B. Aires,1,1,1,1,1,1
B.Aires,1,1,1,1,1,1
Bs As,1,1,1,1,1,1
Bs.As.,1,1,1,1,1,1
Buenos Aires,9,9,9,9,9,9
C deBuenos Aires,1,1,1,1,1,1
CABA,1,1,1,1,1,1
Ciudad de Buenos Aires,3,3,3,3,3,3
Cordoba,1,1,1,1,1,1


In [188]:
tipos_gasto = pd.read_csv('TiposDeGasto.csv')

In [189]:
tipos_gasto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   IdTipoGasto       4 non-null      int64 
 1   Descripcion       4 non-null      object
 2   Monto_Aproximado  4 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 224.0+ bytes


In [190]:
tipos_gasto.head()

,IdTipoGasto,Descripcion,Monto_Aproximado
0,1,Alquiler,1000
1,2,Servicios,300
2,3,Impuestos,100
3,4,Sueldos y Jornales,1000


In [177]:
canaldeventa = pd.read_excel('CanalDeVenta.xlsx')

In [178]:
canaldeventa.head()

,CODIGO,DESCRIPCION
0,1,Telefónica
1,2,OnLine
2,3,Presencial


In [37]:
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
cant_clientes_loc = clientes.groupby('Localidad')['ID'].count()

In [57]:
cant_clientes_loc.sort_values()

Localidad
11 DE SEPTIEMBRE            1
JARDIN ARENALES             1
JOAQUIN GORINA              1
KILOMETRO 15                1
LA ARABIA                   1
                         ... 
MERLO                      35
JOSE C PAZ                 35
BANDFIELD                  36
CORDOBA                   263
CIUDAD DE BUENOS AIRES    458
Name: ID, Length: 379, dtype: int64

In [76]:
def cargar_csv(nombre,ruta,separador,encod):
    nombre = pd.read_csv(ruta,sep=separador,encoding=encod)
    

In [77]:
cant = int(input('Ingrese la cantidad de csv que deasea cargar: '))

for i in range(cant):
    nombre= input('Ingrese el nombre que llevara la tabla')
    ruta = input('Ingrese el path donde esta el archivo')
    separador= input('Indique si tiene algun separador especial')
    encod = input('Indique el encoding que utiliza')
    cargar_csv(nombre=nombre,ruta=ruta,separador=separador,encod=encod)
    print(nombre)
    print('succes')

ventita
succes
